In [7]:
import onnx 
from onnxoptimizer import optimize
import tracemalloc
import timeit

import numpy as np 
import pandas as pd 

import onnxruntime as ort 


In [ ]:
# ts2vec model
# model_run_id: 07406a4af7284df0b8d8f266509161e8
# epoch: 18, model_18

In [54]:
def run_ort(): 
    ort_session = ort.InferenceSession("/Users/fguo/cmt/ts2vec/model_checkpoints/model_18.onnx")
    input_data = np.array([[[-0.5066, -0.0651, -0.1815, -0.8371, -0.5118, -0.0956, -0.2307, -0.4243]]]).astype(np.float32)
    input_name = ort_session.get_inputs()[0].name
    output_name = ort_session.get_outputs()[0].name
    outputs = ort_session.run([output_name], {input_name: input_data})
    ort_out = outputs[0]
    return None 

# 1. Before optimization 

In [55]:
# memory check 
tracemalloc.start()
run_ort()
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
tracemalloc.stop()

Current memory usage is 0.010585MB; Peak was 0.020855MB


2024-03-28 10:14:00.617315 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_if_then_branch__inlfunc_aten_index_put_bool_tmp_7'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:00.617331 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_cond_6'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:00.617335 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_tmp_3'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:00.617339 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_tmp_2'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:00.617343 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initiali

In [56]:
# inference time check 
%timeit run_ort()

2024-03-28 10:14:01.670791 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_if_then_branch__inlfunc_aten_index_put_bool_tmp_7'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:01.670803 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_cond_6'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:01.670808 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_tmp_3'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:01.670812 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_tmp_2'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:01.670815 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initiali

165 ms ± 2.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


AndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_accumulate_as_bool'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:14.589134 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_accumulate'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:14.589146 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_if_then_branch__inlfunc_aten_index_put_bool_token_3_tmp_7'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:14.589149 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_token_3_tmp_3'. It is not used by any node and should be removed from the model.
2024-03-28 10:14:14.589153 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_token_3_cond_6

# After optimization

## (1). Alexnet example

In [43]:
import torch
import torchvision

dummy_input = torch.randn(10, 3, 224, 224, device="cpu")
model = torchvision.models.alexnet(pretrained=True).cpu()
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]

torch.onnx.export(model, dummy_input, "/Users/fguo/cmt/ts2vec/model_checkpoints/alexnet.onnx", verbose=False, 
                  input_names=input_names, output_names=output_names)

/Users/fguo/.pyenv/versions/3.10.13/envs/ts2vec_0/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fguo/.pyenv/versions/3.10.13/envs/ts2vec_0/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
model = onnx.load("/Users/fguo/cmt/ts2vec/model_checkpoints/alexnet.onnx")
# List of optimization passes you want to apply
# Full list: https://github.com/onnx/optimizer/blob/b3a4611861734e0731bbcc2bed1f080139e4988b/onnxoptimizer/pass_registry.h#L39
passes = ["eliminate_unused_initializer", "fuse_bn_into_conv"]
# Apply optimizations
optimized_model = optimize(model, passes)
# Save the optimized model
onnx.save(optimized_model, "/Users/fguo/cmt/ts2vec/model_checkpoints/alexnet_optimized.onnx")

ort_session = ort.InferenceSession("/Users/fguo/cmt/ts2vec/model_checkpoints/alexnet_optimized.onnx")

## (2). ts2vec model

In [60]:
# # This does not work. 
# model = onnx.load("/Users/fguo/cmt/ts2vec/model_checkpoints/model_18.onnx")
# # List of optimization passes you want to apply
# # Full list: https://github.com/onnx/optimizer/blob/b3a4611861734e0731bbcc2bed1f080139e4988b/onnxoptimizer/pass_registry.h#L39
# passes = ["eliminate_nop_dropout"]
# # Apply optimizations
# optimized_model = optimize(model, passes)
# # Save the optimized model
# onnx.save(optimized_model, "/Users/fguo/cmt/ts2vec/model_checkpoints/model_18_optimized.onnx")

# ort_session = ort.InferenceSession("/Users/fguo/cmt/ts2vec/model_checkpoints/model_18_optimized.onnx")

### ORT_ENABLE_ALL

In [57]:
def run_ort_new(): 
    # Set session options for optimization
    sess_options = ort.SessionOptions()
    sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_DISABLE_ALL
    # Create a session with the optimized model
    ort_session = ort.InferenceSession("/Users/fguo/cmt/ts2vec/model_checkpoints/model_18.onnx", sess_options)
    input_data = np.array([[[-0.5066, -0.0651, -0.1815, -0.8371, -0.5118, -0.0956, -0.2307, -0.4243]]]).astype(np.float32)
    input_name = ort_session.get_inputs()[0].name
    output_name = ort_session.get_outputs()[0].name
    outputs = ort_session.run([output_name], {input_name: input_data})
    ort_out = outputs[0]
    return None 

In [58]:
# memory check 
tracemalloc.start()
run_ort_new()
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
tracemalloc.stop()

Current memory usage is 0.002912MB; Peak was 0.013182MB


In [59]:
# inference time check 
%timeit run_ort_new()

70.1 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## QA 

In [61]:
ort_session = ort.InferenceSession("/Users/fguo/cmt/ts2vec/model_checkpoints/model_18.onnx")
input_data = np.array([[[-0.5066, -0.0651, -0.1815, -0.8371, -0.5118, -0.0956, -0.2307, -0.4243]]]).astype(np.float32)
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name
outputs = ort_session.run([output_name], {input_name: input_data})
ort_base = outputs[0]

2024-03-28 10:22:26.644409 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_if_then_branch__inlfunc_aten_index_put_bool_tmp_7'. It is not used by any node and should be removed from the model.
2024-03-28 10:22:26.644827 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_cond_6'. It is not used by any node and should be removed from the model.
2024-03-28 10:22:26.644832 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_tmp_3'. It is not used by any node and should be removed from the model.
2024-03-28 10:22:26.644835 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '_inlfunc_aten_index_put_bool_tmp_2'. It is not used by any node and should be removed from the model.
2024-03-28 10:22:26.644839 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initiali

In [63]:
sess_options = ort.SessionOptions()
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_DISABLE_ALL
# Create a session with the optimized model
ort_session = ort.InferenceSession("/Users/fguo/cmt/ts2vec/model_checkpoints/model_18.onnx", sess_options)
input_data = np.array([[[-0.5066, -0.0651, -0.1815, -0.8371, -0.5118, -0.0956, -0.2307, -0.4243]]]).astype(np.float32)
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name
outputs = ort_session.run([output_name], {input_name: input_data})
ort_new = outputs[0]

In [64]:
close_out = np.isclose(ort_base, ort_new)

In [65]:
ort_base[~close_out]

array([-0.00055191], dtype=float32)

In [66]:
ort_new[~close_out]

array([-0.00055193], dtype=float32)